In [ ]:
# Cell 1: Imports
import pandas as pd
import numpy as np

# Cell 2: Load raw data
df = pd.read_csv("data/raw_dataset.csv", parse_dates=["Date"])

# Cell 3: Initial overview
df.info()
df.describe(include="all")

# Cell 4: Missing data handling
df['Revenue'] = pd.to_numeric(df['Revenue'], errors='coerce')
df['Expenditure'] = pd.to_numeric(df['Expenditure'], errors='coerce')
df['Revenue'].fillna(df.groupby('State')['Revenue'].transform('mean'), inplace=True)
df['Expenditure'].fillna(df.groupby('State')['Expenditure'].transform('median'), inplace=True)

# Cell 5: Outlier detection and conversion
q_low = df['Revenue'].quantile(0.01)
q_high = df['Revenue'].quantile(0.99)
df.loc[df['Revenue'] < q_low, 'Revenue'] = np.nan
df.loc[df['Revenue'] > q_high, 'Revenue'] = np.nan
df['Revenue'].fillna(df['Revenue'].mean(), inplace=True)

# Cell 6: Feature engineering
df['Month'] = df['Date'].dt.month.astype('category')
df['Revenue_per_expenditure'] = df['Revenue'] / df['Expenditure']
df['RevLag1'] = df.groupby('State')['Revenue'].shift(1)
df['RevLag1'].fillna(df['Revenue'].mean(), inplace=True)

# Cell 7: Schema normalization
df_pivot = df.pivot(index='Date', columns='State', values='Revenue').reset_index()

# Cell 8: Save cleaned data
df.to_csv("data/cleaned_dataset.csv", index=False)